1.0 Applying Text-to-Column for all csv files + Saving 

- processed_{orginal filename}
    - processed_S01_1001-1.csv to processed_S01_1001-8.csv
    - processed_S01_1002-1.csv to processed_S01_1002-8.csv 
    - processed_S01_1003-1.csv to processed_S01_1003-9.csv

In [5]:
import pandas as pd
import os

# Specify the directory containing the CSV files
directory = 'AU_60frames_CSV\AU_60frames_CSV'

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Read the entire file into a DataFrame assuming it's a single column with tab-separated values
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Filter out empty lines to avoid processing empty rows
            lines = [line for line in lines if line.strip()]
            # Split each line into columns based on tab
            data = [line.split('\t') for line in lines]

        # Create a DataFrame from the list of lists
        data_expanded = pd.DataFrame(data)

        # Assuming the first row contains headers, set it as the column names
        if data_expanded.shape[0] > 1:
            data_expanded.columns = data_expanded.iloc[0]
            data_expanded = data_expanded[1:]
        
        # Reset the index of the DataFrame
        data_expanded.reset_index(drop=True, inplace=True)

        # Save the modified DataFrame to a new CSV file
        new_file_path = os.path.join(directory, f'processed_{filename}')
        data_expanded.to_csv(new_file_path, index=False)

        print(f'Processed {filename}')

print('All files have been processed.')


Processed S01_1001-1.csv
Processed S01_1001-2.csv
Processed S01_1001-3.csv
Processed S01_1001-4.csv
Processed S01_1001-5.csv
Processed S01_1001-6.csv
Processed S01_1001-7.csv
Processed S01_1001-8.csv
Processed S01_1002-1.csv
Processed S01_1002-2.csv
Processed S01_1002-3.csv
Processed S01_1002-4.csv
Processed S01_1002-5.csv
Processed S01_1002-6.csv
Processed S01_1002-7.csv
Processed S01_1002-8.csv
Processed S01_1003-1.csv
Processed S01_1003-2.csv
Processed S01_1003-3.csv
Processed S01_1003-4.csv
Processed S01_1003-5.csv
Processed S01_1003-6.csv
Processed S01_1003-7.csv
Processed S01_1003-8.csv
Processed S01_1003-9.csv
Processed S01_2001-1.csv
Processed S01_2001-2.csv
Processed S01_2001-3.csv
Processed S01_2001-4.csv
Processed S01_2001-5.csv
Processed S01_2001-6.csv
Processed S01_2001-7.csv
Processed S01_2001-8.csv
Processed S01_2002-1.csv
Processed S01_2002-2.csv
Processed S01_2002-3.csv
Processed S01_2002-4.csv
Processed S01_2002-5.csv
Processed S01_2002-6.csv
Processed S01_2002-7.csv


Aggregating the Action unit for each csv 
1. Read each processed CSV file.
2. Group the data by each Action_Unit column.
3. Sum the Frame_count for each Action_Unit.
4. Count the number of occurrences for each Action_Unit.
5. Save the aggregated results into a new CSV file in a new directory.

In [18]:
import pandas as pd
import os

def process_files(directory):
    # Prepare to store aggregated dataframes for later merging
    aggregated_dataframes = []
    
    # Process each file in the specified directory
    for filename in os.listdir(directory):
        if filename.startswith('processed_') and filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            data = pd.read_csv(filepath)

            # Strip whitespace from column names and print them to debug
            data.columns = data.columns.str.strip()
            print("Cleaned column names:", data.columns)  # Debugging print

            # Perform aggregation
            try:
                aggregated_data = data.groupby('Action_Unit').agg(
                    Frame_count_sum=('Frame_count', 'sum'),
                    Occurrences=('Action_Unit', 'count')
                ).reset_index()
            except KeyError as e:
                print("KeyError:", e)
                continue  # Skip this file if there's a key error

            # Modify the filename index to match video clip filename format
            filename_index = filename.replace('processed_', '').replace('.csv', '.mp4')
            aggregated_data['filename_index'] = filename_index

            # Append the result to the list
            aggregated_dataframes.append(aggregated_data)

    # Combine all dataframes into one
    final_aggregated_df = pd.concat(aggregated_dataframes, ignore_index=True)

    # Load the 'sliced_video_info' file and merge
    sliced_video_info = pd.read_csv('sliced_video_info.csv')
    sliced_video_info.columns = sliced_video_info.columns.str.strip()
    merged_result = pd.merge(sliced_video_info, final_aggregated_df, left_on='Video clip filename', right_on='filename_index')

    # Save the merged result to a new CSV file
    merged_result.to_csv('merged_output.csv', index=False)

    print('Processing and merging complete. Results saved to merged_output.csv.')

# Example usage
process_files('AU_60frames_CSV\AU_60frames_CSV')



Cleaned column names: Index(['No', 'Action_Unit', 'Action_Unit_description', 'Start_time(sec)',
       'End_time(sec)', 'Duration', 'Start_frame', 'End_frame', 'Frame_count',
       'Verify/Refute', 'Comments'],
      dtype='object')
Cleaned column names: Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'], dtype='object')
KeyError: 'Action_Unit'
Cleaned column names: Index(['No', 'Action_Unit', 'Action_Unit_description', 'Start_time(sec)',
       'End_time(sec)', 'Duration', 'Start_frame', 'End_frame', 'Frame_count',
       'Verify/Refute', 'Comments'],
      dtype='object')
Cleaned column names: Index(['No', 'Action_Unit', 'Action_Unit_description', 'Start_time(sec)',
       'End_time(sec)', 'Duration', 'Start_frame', 'End_frame', 'Frame_count',
       'Verify/Refute', 'Comments'],
      dtype='object')
Cleaned column names: Index(['No', 'Action_Unit', 'Action_Unit_description', 'Start_time(sec)',
       'End_time(sec)', 'Duration', 'Start_frame', 'End_frame', 'Frame_count